In [1]:
import numpy as np
# constant size
size = 10000
# Set p(H) apriori for simulation as a biased coin.
p_h = 0.7

# Model experiment as a single biased coin flipped 1000 times.
data = np.random.binomial(1, p_h, size=size)

In [2]:
n_states = 2
n_obs = data.shape[0]

emission = np.random.rand(n_obs,n_states)
transition = np.random.rand(n_states,n_states)
backward = np.random.rand(size,n_states)
backward[-1,:] = 1

forward = np.random.rand(size,n_states)


init_prob = np.random.rand(n_states)
init_obs = np.zeros(n_states)

In [ ]:
for t in reversed(range(size-1)):
    for j in range(n_states):
        backward[t,j] = np.sum(backward[t+1,:]*transition[j,:] * emission[t+1,:])
init_obs = init_prob * emission[0,:] * backward[0,:]

In [7]:
forward[0,:] = init_prob * emission[0,:]
for t in range(size-1):
    forward[t+1,:] = np.matmul(forward[t,:] ,transition) * emission[t+1,:]


In [ ]:
transition_est = np.random.rand(size,n_states,n_states)
for t in range(size):
    for i in range(n_states):
        for j in range(n_states):
            transition_est[t,i,j] = 